# `torch.nn.Embedding`
`torch.nn.Embedding(num_embeddings, embedding_dim, padding_idx=None, max_norm=None, norm_type=2.0, scale_grad_by_freq=False, sparse=False, _weight=None, device=None, dtype=None)`

A simple lookup table that stores embeddings of a fixed dictionary and size.

This module is often used to store word embeddings and retrieve them using indices. The input to the module is a list of indices, and the output is the corresponding word embeddings.

- `num_embeddings` (int) – size of the dictionary of embeddings
- `embedding_dim` (int) – the size of each embedding vector

Obtained from - https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html (For the rest of the parameters description, refer to this link)

In [1]:
import torch
from torch import nn

In [2]:
# n_embeddings can be understood as an equivalent of the size of the vocabulary
# embedding_dim is the number of dimensions for the embedding vector for all the n_embeddings
# In net, it creates an Embedding Matrix with n_embeddings rows and embedding_dim columns
# To access any vector, a one-hot encoded vector is multiplied generally to its transpose 
n_embeddings, embedding_dim = 10, 4

In [4]:
emb_1 = nn.Embedding(n_embeddings, embedding_dim)
emb_1

Embedding(10, 4)

In [5]:
# To view the Embedding Matrix -
emb_1.weight

Parameter containing:
tensor([[-0.1980, -0.9576,  0.8451,  0.4688],
        [ 1.9762,  0.3950,  0.1159,  0.0785],
        [ 0.2741,  0.5328,  0.8331, -1.1398],
        [-1.4993, -1.1635,  0.4245, -1.0756],
        [-0.7719,  1.3352, -1.0978,  0.0203],
        [-1.3495, -0.2971, -1.8949, -0.0158],
        [-0.9143,  1.0154,  0.3341,  1.8463],
        [-0.8330,  1.6009, -0.8548, -0.4937],
        [-0.2467, -0.0735, -0.6596,  0.6586],
        [ 0.0241, -1.6811, -0.7133,  0.1182]], requires_grad=True)

This array is by default learnable as `requires_grad = True`

In [14]:
inp = torch.LongTensor([[1 ,3], [5, 5]])
emb_1(inp) # Forward Pass

tensor([[[ 1.9762,  0.3950,  0.1159,  0.0785],
         [-1.4993, -1.1635,  0.4245, -1.0756]],

        [[-1.3495, -0.2971, -1.8949, -0.0158],
         [-1.3495, -0.2971, -1.8949, -0.0158]]], grad_fn=<EmbeddingBackward0>)

Returns the respective rows in the embedding matrix

In [10]:
inp2 = torch.LongTensor([2])
emb_1(inp2) # Forward Pass

tensor([[ 0.2741,  0.5328,  0.8331, -1.1398]], grad_fn=<EmbeddingBackward0>)

In [8]:
emb_1.weight[2]

tensor([ 0.2741,  0.5328,  0.8331, -1.1398], grad_fn=<SelectBackward0>)

## `padding_idx`

In [12]:
emb_2 = nn.Embedding(n_embeddings, embedding_dim, padding_idx=5)
emb_2.weight

Parameter containing:
tensor([[ 0.1703,  0.3528,  0.2223, -0.5510],
        [ 0.2252,  0.4571, -0.1718, -0.3691],
        [ 0.0319, -1.9599, -1.5053, -1.0299],
        [ 0.9450,  0.1584,  0.7127,  0.9653],
        [ 0.5396, -1.1378, -0.9376, -0.8273],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 1.1557,  0.3902, -0.1632,  1.1344],
        [-0.8179,  1.4932,  1.4569,  1.6835],
        [ 0.8030,  0.2815,  0.4483, -0.8984],
        [-3.4135,  1.1419,  1.5185, -2.1653]], requires_grad=True)

We can see that the 5th index (6th row) row is all zeros

In [16]:
emb_2(inp).mean().backward() # Backprop
emb_2.weight.grad

tensor([[0.0000, 0.0000, 0.0000, 0.0000],
        [0.0625, 0.0625, 0.0625, 0.0625],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [0.0625, 0.0625, 0.0625, 0.0625],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000]])

We can see that the 5th index row (6th row) is still all zeros, hence **`padding_idx` ensures WHICH word in the vocabulary (or can be in something else) must NOT UPDATE its Embeddings or not care about their embeddings.**

## `norm`
Goal is to make sure the norm of the embeddings is not TOO big or in other words, that the embeddings do not move too far from the origin.

`norm_type`: Setting p value for l_p norm

`max_norm`: Setting maximum value of the l_p norm

In [18]:
# norm_type = 2 means l2 norm
emb_3 = nn.Embedding(n_embeddings, embedding_dim, norm_type=2, max_norm=1)
emb_3.weight

Parameter containing:
tensor([[ 1.8580,  1.7609, -1.7272, -0.2996],
        [ 0.9081, -0.2909,  1.4354,  1.6269],
        [-0.0481,  0.9890, -0.3541,  0.5335],
        [ 0.5122,  1.2307, -0.0887,  0.3569],
        [-0.5391, -0.3979,  0.2598,  0.7407],
        [ 2.0631,  0.7755, -0.7071,  1.3374],
        [-0.5664,  2.1410, -0.3379, -1.6817],
        [ 1.4850, -0.8659, -0.4306,  0.9667],
        [-0.4673, -0.1302, -0.0203,  0.0969],
        [-2.3174,  1.6792, -0.9698, -0.8020]], requires_grad=True)

In [19]:
emb_3.weight.norm(dim=-1) # Returns row-wise norm (Norm of every row in embedding matrix)

tensor([3.1026, 2.3699, 1.1791, 1.3829, 1.0320, 2.6733, 2.8013, 2.0187, 0.4951,
        3.1262], grad_fn=<NormBackward1>)

We can see that the normalization of rows *did not happen*. For this one must first run the forward pass

In [20]:
emb_3(inp).norm(dim=-1) # Inputs included values 1,3,5

tensor([[1.0000, 1.0000],
        [1.0000, 1.0000]], grad_fn=<NormBackward1>)

In [21]:
emb_3.weight.norm(dim=-1)

tensor([3.1026, 1.0000, 1.1791, 1.0000, 1.0320, 1.0000, 2.8013, 2.0187, 0.4951,
        3.1262], grad_fn=<NormBackward1>)

We can now see that **only** the input values which were **passed** in the forward pass, were **normalized** (1st, 3rd and 5th index)

General goal of setting max_norm is for **regularization** and **scaling**